# Nationalratswahlen

In [19]:
import os
from pyaxis import pyaxis
import json
import pandas as pd
import sys

In [20]:
from constants import INDICATOR_ID_MAPPING, URL_MAPPING, SPATIAL_UNIT_ONTOLOGY, TIME_INFO_ID, DATA_FOLDER
from utils import map_name_to_number, save_data

In [21]:
dataset = 'national_council_elections'

In [22]:
# load data from bfs
px = pyaxis.parse(URL_MAPPING[dataset], encoding='ISO-8859-2')
df = px['DATA']

In [23]:
# rename 'Kanton (-) / Bezirk (>>) / Gemeinde (......)' to 'name_de'
df = df.rename(columns={
    'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'name_de', 
    'Jahr': 'year', 
    'Ergebnisse': 'results', 
    'DATA': 'data'
})

In [24]:
df['name_de'] = df['name_de'].str.replace('.', '')
df['name_de'] = df['name_de'].str.replace('>> ', '')
df['name_de'] = df['name_de'].str.replace('   ', '')

/tmp/ipykernel_1050/2170717606.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['name_de'] = df['name_de'].str.replace('.', '')


In [25]:
df.head()

,name_de,year,results,data
0,Schweiz,2019,Wahlberechtigte,5459218
1,Schweiz,2019,Wählende / Eingelegte Wahlzettel (WZ),2462641
2,Schweiz,2019,Wahlbeteiligung [%],45.1097758
3,Schweiz,2019,Leere WZ,9366
4,Schweiz,2019,Ungültige WZ,29015


In [26]:
df, metadata = map_name_to_number(df, columns_to_map=['results'])

In [27]:
metadata

{'results': {0: 'Ungültige WZ',
  1: 'Listenstimmen aus veränderten WZ',
  2: 'Veränderte WZ',
  3: 'Total Kandidatenstimmen',
  4: 'Total Zusatzstimmen',
  5: 'Wahlbeteiligung [%]',
  6: 'Wahlberechtigte',
  7: 'Leere WZ',
  8: 'Gültige WZ',
  9: 'WZ ohne Listenbezeichnung',
  10: 'Zusatzstimmen aus unveränderten WZ',
  11: 'Zusatzstimmen aus veränderten WZ',
  12: 'Listenstimmen aus unveränderten WZ',
  13: 'Kandidatenstimmen aus unveränderten WZ',
  14: 'Kandidatenstimmen aus veränderten WZ',
  15: 'Unveränderte WZ',
  16: 'Total Listenstimmen',
  17: 'Wählende / Eingelegte Wahlzettel (WZ)'}}

In [28]:
# create new dataframe
df_new = pd.DataFrame({
    'indicator_id': INDICATOR_ID_MAPPING[dataset],
    'spatialunit_name': df['name_de'],
    'spatialunit_id': 'TODO', # TODO once list is done
    'time_value': df['year'],
    'timeinfo_id': TIME_INFO_ID['REFERENCE_DAY'],
    'results': df['results'],
    'value': df['data']
})

In [29]:
df_new.head()

,indicator_id,spatialunit_name,spatialunit_id,time_value,timeinfo_id,results,value
0,1_10_101,Schweiz,TODO,2019,1,6,5459218
1,1_10_101,Schweiz,TODO,2019,1,17,2462641
2,1_10_101,Schweiz,TODO,2019,1,5,45.1097758
3,1_10_101,Schweiz,TODO,2019,1,7,9366
4,1_10_101,Schweiz,TODO,2019,1,0,29015


In [32]:
save_data(df_new, metadata, dataset_name = 'national_council_elections')